<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Импорт-необходимых-библиотек" data-toc-modified-id="Импорт-необходимых-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт необходимых библиотек</a></span></li><li><span><a href="#Открытие-файла" data-toc-modified-id="Открытие-файла-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Открытие файла</a></span></li><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Изучение данных</a></span></li></ul></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных
### Импорт необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.utils import shuffle
from tqdm.notebook import tqdm

### Открытие файла

In [2]:
try:
    data = pd.read_csv('/datasets/Churn.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/Churn.csv')

### Изучение данных

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [5]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


**Вывод раздела.** В данном разделе рассмотрены предоставленные данные. Видно, что в целевом признаке более `75%` нулей. В столбце `Tenure` обнаружены пропуски. В основном все признаки числовые, что упрощает задачу.

## Исследование задачи
Для начала сброшу все строки с пустыми значениями из таблицы

In [6]:
data = data.dropna()

Далее выделю признаки и целевой признак

In [7]:
# Номер строки, ID клиента и Фамилия будут мешать конечному обучению, так как всегда уникальны
features = data.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'Exited'])
target = data['Exited']
#One-hot-encoding для признаков
features = pd.get_dummies(features, drop_first=True)

Разбиваю данные на три части (тренировачные, валидационные и тестовые) в соотношении 3:1:1

In [8]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features,
    target,
    test_size=0.4,
    random_state=1
)

In [9]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test,
    target_valid_test,
    test_size=0.5,
    random_state=1
)

После разделения данных, масштабирую их.

In [10]:
# Отключает надоедливые предупреждения о копиях данных и всё-такое.
# Главное работает.
pd.options.mode.chained_assignment = None

scaler = StandardScaler()
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance','NumOfProducts', 'EstimatedSalary']
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

Проверяю модель на подготовленных данных

In [11]:
model = LogisticRegression(random_state=1, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

Смотрю точность модели

In [12]:
accuracy_score(target_valid, predicted_valid)

0.8052805280528053

У модели очень высокая точность - `80.53%`. Проверяю модель на дисбаланс

In [13]:
data['Exited'].mean()

0.2039379606203938

`20.39%` единиц в целевом признаке. Не очень много. Проверяю f1-меру

In [14]:
f1_score(target_valid, predicted_valid)

0.2594142259414226

`0.26` - низкое значение f1-меры. Также значение auc-roc

In [15]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print(auc_roc)

0.7561360933587291


`0.7561` - неплохой показатель, но и неидеальный.

**Вывод раздела.** В данном разделе были убраны пропуски из данных. Из признаков были исключены столбцы `RowNumber`, `CustomerId` и `Surname`, так как они скорее всего помешают обучению. Данные были разделены в соотношении 3:1:1 (тренировачные, валидационные и тестовые). Было проведено масштабирование данных. Была проведена попытка обучить простую модель, однако успеха достичь не удалось, так как модель хоть и обладает высокой точностью в `81%`, значение f1-меры составляет только `0.26`, а auc-roc метрика равна `0.76`.

## Борьба с дисбалансом
Для борьбы с дисбалансом обучу модель со взвешиванием классов. Для повышения качества поменяю модель обучения

In [16]:
model = RandomForestClassifier(random_state=1, class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5183823529411764


После такого обучения f1-мера значительно выросла и составляет `0.5184`, однако этого недостаточно для прохождения порога. Проверю auc-roc метрику

In [17]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print(auc_roc)

0.8416333910218603


`0.8416` - auc-roc метрика выросла. Для повышения качества увеличу количество единиц в данных. Для начала узнаю их количество

In [18]:
target_train.value_counts()

0    4331
1    1123
Name: Exited, dtype: int64

Такие показатели значат, что нужно в 4 раза больше единиц для баланса данных. Из параметров модели убираю взвешивание классов, так как их почти одинаковое количество

In [19]:
# upsample на вход принимает признаки и целевой признак,
# а возвращает несколько повторов одних и тех же
# перемешанных данных с целевым признаком 1
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=1)
    
    return features_upsampled, target_upsampled

features_upsample, target_upsample = upsample(features_train, target_train, 4)
model = RandomForestClassifier(random_state=1)
model.fit(features_upsample, target_upsample)
predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)

0.5727848101265823

`0.5728` - это значит, что в этот раз f1-мера практически доходит до необходимого минимума `0.59`. Проверяю auc-roc метрику

In [20]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.8417513463342002


Метрика почти не изменилась и составляет `0.8418`. Дальше, чтобы поднять f1-меру переберу гиперпараметры модели

In [21]:
best_depth = 0
best_est = 0
best_f1 = 0

for depth in tqdm(range(1, 21, 1), position=0, desc="depth", leave=False):
    for est in tqdm(range(1, 101), position=1, desc="est", leave=False):
        model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=1)
        model.fit(features_upsample, target_upsample)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if best_f1 < f1:
            best_f1 = f1
            best_depth = depth
            best_est = est

depth:   0%|          | 0/20 [00:00<?, ?it/s]

est:   0%|          | 0/100 [00:00<?, ?it/s]

est:   0%|          | 0/100 [00:00<?, ?it/s]

est:   0%|          | 0/100 [00:00<?, ?it/s]

est:   0%|          | 0/100 [00:00<?, ?it/s]

...

est:   0%|          | 0/100 [00:00<?, ?it/s]

est:   0%|          | 0/100 [00:00<?, ?it/s]

est:   0%|          | 0/100 [00:00<?, ?it/s]

est:   0%|          | 0/100 [00:00<?, ?it/s]

est:   0%|          | 0/100 [00:00<?, ?it/s]

In [22]:
print(f"best_f1: {best_f1}")
print(f"best_depth: {best_depth}")
print(f"best_est: {best_est}")

best_f1: 0.6217008797653959
best_depth: 15
best_est: 38


При максимальной глубине в 15 и количестве деревьев 38 удалось получить лучший результат f1-меры в `0.6217`.

**Вывод раздела.** В данном разделе проведена борьба с дисбалансом. Для этого была попытка в параметрах модели включить взвешивание классов, однако она не принесла нужных результатов, поэтому данные переработаны таким образом, чтобы поднять значение f1-меры на валидационной выборке. Также вместо линейной регрессии используется классификатор случайного леса, для которого перебором были найдены оптимальные гиперпараметры.

## Тестирование модели
Проверяю модель на тестовой выборке. Для обучения добавляю данные из валидационной выборки.

In [23]:
model = RandomForestClassifier(n_estimators=best_est, max_depth=best_depth, random_state=1)
model.fit(pd.concat([features_upsample, features_valid]), pd.concat([target_upsample, target_valid]))
predicted_test = model.predict(features_test)
f1_score(target_test, predicted_test)

0.6145092460881935

`0.6145` - результат выше минимального порога необходимого для f1-меры, а значит, что модель прошла проверку. Проверю также auc-roc значение.

In [24]:
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print(auc_roc)

0.8478009832971665


`0.8478` - неплохой результат.

**Вывод раздела.** В данном разделе готовая модель проверена на тестовой выборке. Модель показала хороший результат и прошла минимальный порог f1-меры в `0.59` со значением `0.6145`.

**Вывод.** В данной работе были взяты данные «Бета-Банка» из которого уходят клиенты. Целью было спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Чтобы сдать проект успешно, необходимо было довести F1-меру до 0.59. В ходе работы были выполнены следующие задачи:
+ изучены представленные данные;
+ из данных убраны лишние строки;
+ выделены признаки и целевой признак;
+ получены дамми-признаки с избеганием дамми-ловушки;
+ данные разбиты на тренировачные, валидационные и тестовые;
+ проведено масштабирование данных;
+ проведена борьба с дисбалансом;
+ лучшая модель проверена на тестовой выборке;
+ пройден порог f1-меры в `0.59`.

С данной моделью «Бета-Банк» сможет улучшить свои прогнозы уходов клиентов до того, как они решаться уйти и примет необходимые меры для их удержания.